In [1]:
import torch
import argparse
import time
import math
import os, sys
import itertools
import pickle
from tqdm import tqdm
import numpy as np
import os.path as osp

In [2]:
torch.cuda.device_count()

1

In [4]:
def loadpkl(path):
    with open(path, 'rb') as f:
        rst=pickle.load(f)
    return rst

In [5]:
path="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml/full_id_abs_title_cite_ref_author.pkl"
full_data=loadpkl(path)

In [6]:
path="/share/data/mei-work/kangrui/github/ssref/result/eval_rerank/test_result/eval_key_id2embed.pkl"
key_embedding=loadpkl(path)

In [17]:
path="/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/f1_result/eval_test_pred_1000.pkl"
pred_set=loadpkl(path)

In [18]:
path="/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/f1_result/eval_test_gold.pkl"
gold_set=loadpkl(path)

In [19]:
query_id2embedding=key_embedding
key_id2embedding=key_embedding

In [20]:
candidate_set={}
candidate_ref_set={}
for k,v in pred_set.items():
    cur=set()
    for paper in v[:200]:
        cur.add(paper)
        refset=set()
        for ref in full_data[paper]["references"]:
            cur.add(ref)
            refset.add(ref)
        candidate_ref_set[paper]=refset
    candidate_set[k]=list(cur)

In [21]:
tensor = torch.randn((5, 10))
print(tensor)

tensor([[-1.9392,  0.9045, -0.0835,  0.2560, -1.6710,  0.1556,  0.4475, -1.1248,
          0.7270,  0.3560],
        [-0.0942,  0.8383,  0.5340, -0.7872, -0.3984, -1.2047,  0.3545,  0.5832,
          2.1999, -0.3582],
        [-0.2738, -0.5424, -0.2009, -0.6824,  0.4770, -0.9267,  0.5580, -0.7867,
         -1.3659, -1.8759],
        [ 0.0259, -1.2168,  0.7497,  0.5638,  1.1777,  0.3550,  0.1802, -0.4498,
          0.3754,  0.0600],
        [ 0.4391, -0.3672,  1.8396,  0.7395,  0.7778,  0.5637,  0.3116,  0.3939,
         -0.9402, -1.2173]])


In [22]:
mean_tensor = torch.mean(tensor, dim=0)
print(mean_tensor.shape)

torch.Size([10])


In [23]:

    
def rerank(candidate_set,query_id2embedding,key_id2embedding,candidate_ref_set,rerank_num=None,device=None):
    cnt=0
    pbar = tqdm(range(0,len(candidate_set),1), postfix=f"calculating f1")
    query=list(candidate_set.keys())
    empty_embed=torch.tensor(np.zeros_like(next(iter(key_id2embedding.values()))))
    empty_embed_cnt=0
    if rerank_num is None:
        for v in candidate_set.values():
            if rerank_num is None or rerank_num>len(v):
                rerank_num=len(v)
                
    rerank_candidate={}   
    for i in pbar:
        query_paper=query[i]
        query_embedding=torch.tensor(query_id2embedding[query_paper]).unsqueeze(0)
        
        cited_dict={}
        for candidate in candidate_set[query_paper]:
            
            if candidate in candidate_ref_set.keys():
                # this is possibly the anchor paper
                for ref in list(candidate_ref_set[candidate]):
                    # deal with the ref of an anchor paper
                    if ref in candidate_set[query_paper]:
                        if ref not in cited_dict.keys():
                            cited_dict[ref]=[candidate]
                        else:
                            cited_dict[ref].append(candidate)
                    else:
                        # this is not anchor paper for this query
                        break
            if candidate not in cited_dict.keys():
                cited_dict[candidate]=[candidate]
            else:
                cited_dict[candidate].append(candidate)
                
                
        key_embedding=[]
        for candidate in candidate_set[query_paper]:
            candidate_embedding=[]
            for paper in cited_dict[candidate]:
                if paper in key_id2embedding:
                    candidate_embedding.append(torch.tensor(key_id2embedding[paper]))
            if len(candidate_embedding)==0:
                candidate_embedding.append(empty_embed)
                empty_embed_cnt+=1
            candidate_embedding=torch.sum(torch.stack(candidate_embedding),dim=0)
            
            key_embedding.append(candidate_embedding)
        
        
        key_embedding=torch.stack(key_embedding)
        query_embedding=query_embedding.to(device)
        key_embedding=key_embedding.to(device)
        pred_logits=torch.mm(query_embedding,key_embedding.T)
        top_k=torch.topk(pred_logits[0],k=rerank_num)[1]
#         print(pred_logits[0][top_k[-5:]])
        rerank=[candidate_set[query_paper][idx.cpu().item()] for idx in top_k]
        rerank_candidate[query_paper]=rerank
    print("empty embedding cnt->",empty_embed_cnt)
    print("min rerank num->",rerank_num)
    return rerank_candidate
        
        


In [24]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [25]:
rerank_candidate=rerank(candidate_set,query_id2embedding,key_id2embedding,candidate_ref_set,device=device)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 944/944 [10:47<00:00,  1.46it/s, calculating f1]

empty embedding cnt-> 0
min rerank num-> 1891


In [26]:
sys.path.append("/share/data/mei-work/kangrui/github/ssref")
from utils.test_utils import *
from utils.utils import *

In [27]:
def show_rerank_rst(rerank_candidate,gold_set):
    pool_length=len(next(iter(rerank_candidate.values())))
    i=2
    while 1:
        all_pred_topN=[]
        all_gold=[]
        for query_paper,paperlist in rerank_candidate.items():
            topN=paperlist[:i]
            all_pred_topN.append(topN)
            all_gold.append(gold_set[query_paper])
        all_pred_topN=np.array(all_pred_topN)
        metric = get_precision_recall_f1(all_pred_topN, all_gold)
        print(f"top{i}:\n{metric}")
        if i>pool_length:
            break
        i*=2

In [28]:
show_rerank_rst(rerank_candidate,gold_set)

top2:
{'precision': 0.2605932203389831, 'recall': 0.012799167533818939, 'f1': 0.024399920650664552, 'true_pos': 492, 'false_pos': 1396, 'false_neg': 37948, 'num_sample': 944}
top4:
{'precision': 0.2955508474576271, 'recall': 0.02903225806451613, 'f1': 0.052870949403069925, 'true_pos': 1116, 'false_pos': 2660, 'false_neg': 37324, 'num_sample': 944}
top8:
{'precision': 0.2856197033898305, 'recall': 0.056113423517169615, 'f1': 0.09379892155157418, 'true_pos': 2157, 'false_pos': 5395, 'false_neg': 36283, 'num_sample': 944}
top16:
{'precision': 0.23689088983050846, 'recall': 0.09308012486992716, 'f1': 0.1336470939787838, 'true_pos': 3578, 'false_pos': 11526, 'false_neg': 34862, 'num_sample': 944}
top32:
{'precision': 0.1855137711864407, 'recall': 0.14578563995837668, 'f1': 0.16326768441906536, 'true_pos': 5604, 'false_pos': 24604, 'false_neg': 32836, 'num_sample': 944}
top64:
{'precision': 0.13504700741525424, 'recall': 0.2122528616024974, 'f1': 0.16506838229343693, 'true_pos': 8159, 'false